In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Veri seti yolu
dataset_dir = '/home/onuryilmazo/Desktop/patates/data'

# Veri ön işleme ve artırma
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8,1.2],
    fill_mode='nearest',
    validation_split=0.2)  # Doğrulama seti için %20 ayrılıyor

# Eğitim ve doğrulama veri generator'larını oluşturma
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Eğitim için veri seti

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Doğrulama için veri seti

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    BatchNormalization(),  # Yeni: Batch normalization
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),  # Yeni: Batch normalization
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),  # Yeni: Batch normalization
    Conv2D(128, (3, 3), activation='relu'),  # Yeni: Ekstra bir katman
    MaxPooling2D(2, 2),
    BatchNormalization(),  # Yeni: Batch normalization
    Flatten(),
    Dropout(0.5),  # Düzenlileştirme için Dropout
    Dense(512, activation='relu'),
    BatchNormalization(),  # Yeni: Batch normalization
    Dense(7, activation='softmax')  # 7 sınıf için çıkış katmanı
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Modeli eğitin
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Found 363 images belonging to 7 classes.
Found 88 images belonging to 7 classes.
Epoch 1/25
11/11 [==============================] - 3s 245ms/step - loss: 2.6392 - accuracy: 0.3505 - val_loss: 1.9319 - val_accuracy: 0.2188
Epoch 2/25
11/11 [==============================] - 2s 203ms/step - loss: 2.4679 - accuracy: 0.3565 - val_loss: 2.7940 - val_accuracy: 0.1875
Epoch 3/25
11/11 [==============================] - 3s 229ms/step - loss: 2.0016 - accuracy: 0.3988 - val_loss: 3.4743 - val_accuracy: 0.1875
Epoch 4/25
11/11 [==============================] - 3s 223ms/step - loss: 1.6003 - accuracy: 0.4471 - val_loss: 2.5851 - val_accuracy: 0.1875
Epoch 5/25
11/11 [==============================] - 2s 207ms/step - loss: 1.7633 - accuracy: 0.3867 - val_loss: 2.9286 - val_accuracy: 0.2031
Epoch 6/25
11/11 [==============================] - 3s 244ms/step - loss: 1.6239 - accuracy: 0.4290 - val_loss: 2.8892 - val_accuracy: 0.2344
Epoch 7/25
11/11 [==============================] - 3s 266ms/step -

Epoch 25/25
11/11 [==============================] - 2s 216ms/step - loss: 1.3329 - accuracy: 0.4985 - val_loss: 1.5016 - val_accuracy: 0.3750

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Veri setinin %20'si doğrulama için ayrılacak
)

# Eğitim ve doğrulama veri generator'larını oluşturma
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Eğitim için veri seti
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Doğrulama için veri seti
)


Found 363 images belonging to 7 classes.
Found 88 images belonging to 7 classes.


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Aşırı uyumu azaltmak için dropout katmanı
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Erken durdurma ve en iyi modeli kaydetmek için callback'ler
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,  # Daha fazla epoch, erken durdurma kullanıldığı için sorun olmamalı
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/100
11/11 [==============================] - 4s 247ms/step - loss: 2.1166 - accuracy: 0.1239 - val_loss: 1.9178 - val_accuracy: 0.2969
Epoch 2/100


/home/onuryilmazo/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 3s 259ms/step - loss: 1.8725 - accuracy: 0.2417 - val_loss: 1.7997 - val_accuracy: 0.3281
Epoch 3/100
11/11 [==============================] - 3s 234ms/step - loss: 1.7812 - accuracy: 0.2659 - val_loss: 1.6867 - val_accuracy: 0.2812
Epoch 4/100
11/11 [==============================] - 3s 254ms/step - loss: 1.6806 - accuracy: 0.3011 - val_loss: 1.6810 - val_accuracy: 0.3594
Epoch 5/100
11/11 [==============================] - 3s 240ms/step - loss: 1.6285 - accuracy: 0.3202 - val_loss: 1.6772 - val_accuracy: 0.2656
Epoch 6/100
11/11 [==============================] - 3s 245ms/step - loss: 1.6414 - accuracy: 0.3293 - val_loss: 1.6291 - val_accuracy: 0.2812
Epoch 7/100
11/11 [==============================] - 3s 251ms/step - loss: 1.6097 - accuracy: 0.3414 - val_loss: 1.5846 - val_accuracy: 0.3906
Epoch 8/100
11/11 [==============================] - 3s 244ms/step - loss: 1.5570 - accuracy: 0.3595 - val_loss: 1.5431 - val_accuracy: 0.3438
Epoch 9/100

In [11]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D

# Önceden eğitilmiş bir ResNet50 modeli yükleyin
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(150, 150, 3)))

# Modelin üst katmanlarını dondurun
for layer in base_model.layers:
    layer.trainable = False

# Özel üst katmanlar ekleyin
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)  # 7 sınıf için çıkış katmanı

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

94765736/94765736 [==============================] - 71s 1us/step


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100
11/11 [==============================] - 7s 521ms/step - loss: 2.1339 - accuracy: 0.1360 - val_loss: 2.0729 - val_accuracy: 0.1719
Epoch 2/100
11/11 [==============================] - 5s 406ms/step - loss: 2.0169 - accuracy: 0.1782 - val_loss: 2.0202 - val_accuracy: 0.1094
Epoch 3/100
11/11 [==============================] - 5s 423ms/step - loss: 1.9974 - accuracy: 0.1761 - val_loss: 2.0277 - val_accuracy: 0.2344
Epoch 4/100
11/11 [==============================] - 5s 418ms/step - loss: 2.0043 - accuracy: 0.1631 - val_loss: 2.0064 - val_accuracy: 0.1875
Epoch 5/100
11/11 [==============================] - 5s 423ms/step - loss: 1.9640 - accuracy: 0.1571 - val_loss: 1.9822 - val_accuracy: 0.1406
Epoch 6/100
11/11 [==============================] - 5s 408ms/step - loss: 1.9374 - accuracy: 0.2447 - val_loss: 1.8961 - val_accuracy: 0.2500
Epoch 7/100
11/11 [==============================] - 4s 421ms/step - loss: 1.9375 - accuracy: 0.2266 - val_loss: 1.8993 - val_accuracy: 0.2500

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


2024-01-16 16:17:44.991538: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 16:17:45.034642: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 16:17:45.697286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Veri setinin yolu
dataset_dir = '/home/onuryilmazo/Desktop/patates/data'  # Bu yolu veri setinizin konumuna göre güncelleyin

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 363 images belonging to 7 classes.
Found 88 images belonging to 7 classes.


In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Modelin üst katmanlarını dondurun
for layer in base_model.layers:
    layer.trainable = False

# Özel üst katmanları ekleyin
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


2024-01-16 16:18:59.934106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 16:18:59.938419: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 2.2309 - accuracy: 0.1631

/home/onuryilmazo/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 9s 555ms/step - loss: 2.2309 - accuracy: 0.1631 - val_loss: 2.0787 - val_accuracy: 0.1094
Epoch 2/100
11/11 [==============================] - 5s 445ms/step - loss: 2.0942 - accuracy: 0.1360 - val_loss: 1.9729 - val_accuracy: 0.2031
Epoch 3/100
11/11 [==============================] - 5s 429ms/step - loss: 2.0334 - accuracy: 0.1541 - val_loss: 1.9156 - val_accuracy: 0.1875
Epoch 4/100
11/11 [==============================] - 5s 433ms/step - loss: 1.9998 - accuracy: 0.1631 - val_loss: 1.9092 - val_accuracy: 0.2969
Epoch 5/100
11/11 [==============================] - 5s 448ms/step - loss: 2.0005 - accuracy: 0.1782 - val_loss: 1.9028 - val_accuracy: 0.2656
Epoch 6/100
11/11 [==============================] - 4s 396ms/step - loss: 1.9662 - accuracy: 0.1662 - val_loss: 1.9063 - val_accuracy: 0.2656
Epoch 7/100
11/11 [==============================] - 4s 402ms/step - loss: 1.9530 - accuracy: 0.1813 - val_loss: 1.9122 - val_accuracy: 0.2656
Epoch 8/100